In [15]:

import os
import random
import time
from pydantic import BaseModel, Field
from typing import Dict, Any, TypedDict, Literal, List, Optional

from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, END
from langchain.agents import AgentExecutor
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage
from langchain_core.runnables import RunnableParallel
from langchain_core.tools import tool
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.prebuilt import create_react_agent
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool


import re

In [16]:
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")



In [17]:
# Data Storage 
class DataStorage:
    """Simulates a database. In production, replace with Redis, a DB, etc."""
    _storage = {
        'document_content': {},
        'summary_output': {},
        'insights_output': {},
        'action_items': {}
    }
    
    @classmethod
    def store(cls, data_type: str, data: any) -> str:
        """Store data and return a unique reference ID."""
        # Using a simple timestamp and random number for a unique ID
        uid = f"{data_type}_{int(time.time() * 1000)}_{random.randint(100, 999)}"
        cls._storage[data_type][uid] = data
        print(f"📦 Stored data of type '{data_type}' with ID: {uid}")
        return uid
    
    @classmethod
    def retrieve(cls, data_type: str, uid: str) -> any:
        """Retrieve data by its reference ID."""
        print(f" retrievel data of type '{data_type}' with ID: {uid}")
        return cls._storage[data_type].get(uid)

In [4]:

# GraphState 
# The state now holds lightweight IDs instead of large text blobs.
class GraphState(TypedDict):
    """Refactored state using IDs for scalability."""
    document_content_id: str
    summary_status: str
    insights_status: str
    summary_id: Optional[str]
    insights_id: Optional[str]
    action_items_id: Optional[str]
    iteration: int
    error_message: str
    current_reasoning: str
    next: str

In [5]:
# Action Item Extraction Tool 
class ActionItem(BaseModel):
    task: str = Field(description="The specific action or task to be completed.")
    owner: Optional[str] = Field(description="The person or team responsible for the task.")
    deadline: Optional[str] = Field(description="The due date for the task, e.g., 'EOW', '2024-08-15'.")

In [6]:
# Simple but intelligent decision model
class SupervisorDecision(BaseModel):
    """AI Supervisor decision with intelligent reasoning"""
    next_action: Literal[
        "call_both_parallel", 
        "call_summary_only", 
        "call_insights_only", 
        "end_workflow"
    ] = Field(description="What to do next in the workflow")
    
    reasoning: str = Field(description="Why this decision makes sense for the workflow goal")
    
    confidence: float = Field(
        description="Confidence in this decision (0.0 to 1.0)", 
        ge=0.0, le=1.0
    )

In [7]:
@tool
def extract_and_store_action_items(document_content: str) -> str:
    """
    Identifies action items in a document, structures them,
    stores them, and returns a confirmation with the storage ID.
    """
    print("\n🛠️ Action Item Tool Called...")

    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an expert at extracting structured data. Identify all action items from the text. For each, extract the task, owner, and deadline. Use 'N/A' if missing."),
        ("human", "Extract action items from this document:\n\n---\n\n{document}")
    ])
    llm = ChatGroq(temperature=0, model="deepseek-r1-distill-llama-70b")

    # This correctly defines the schema for a list of items
    class ActionItems(BaseModel):
        action_items: List[ActionItem]

    extractor = prompt | llm.with_structured_output(schema=ActionItems)

    try:
        # The result will be an instance of the ActionItems class
        result = extractor.invoke({"document": document_content})

        action_items_list = result.action_items if result and hasattr(result, "action_items") else []
        
        if not action_items_list:
            return "No action items were found in the document."

        # Store the list of Pydantic objects
        storage_id = DataStorage.store('action_items', action_items_list)
        print(f"✅ Action items extracted and stored successfully.")
        
        return f"Confirmation: Successfully stored {len(action_items_list)} action items with ID {storage_id}"

    except Exception as e:
        print(f"❌ Error in Action Item Tool: {e}")
        return "An error occurred during action item extraction."

In [9]:
def intelligent_supervisor(state: GraphState) ->  Dict[str, Any]:
    """
    Smart supervisor that understands the workflow goal:
    - Get meeting minutes summary 
    - Get key insights in parallel
    - Be intelligent about when to retry vs when to finish
    """
    print(f"\n🧠 AI Supervisor thinking... (Iteration: {state['iteration']})")
    
    # Enhanced ReAct-style system prompt
    system_prompt = """
    You are an intelligent workflow supervisor using ReAct (Reasoning + Acting) methodology. 

    ## YOUR MISSION:
    Ensure we successfully get a meeting summary and key insights stored efficiently.

    ## ReAct PROCESS - Think step by step:
    **THOUGHT**: First, analyze the current workflow state. What's working? What failed? Why might it have failed?
    **OBSERVATION**: A task is 'successful' if its status is 'success' AND its ID exists. A task has 'failed' if its status is 'failed'.
    **ACTION**: Based on your thought and observation, decide the smartest next action.

    ## CURRENT WORKFLOW STATE:
    - Summary Status: {summary_status} (summary_id: {summary_id})
    - Insights Status: {insights_status} (insights_id: {insights_id})
    - Current Iteration: {iteration}
    - Last Error: {error_msg}

    ## AVAILABLE ACTIONS:
    - call_both_parallel: Run both agents simultaneously.
    - call_summary_only: Focus only on getting the summary.
    - call_insights_only: Focus only on getting insights.  
    - end_workflow: Use this ONLY when both Summary and Insights have a 'success' status and their IDs exist.

    ## INTELLIGENT DECISION GUIDELINES:
    - Iteration 1: Usually start with parallel execution for efficiency.
    - One success, one failure: Target retry on the failed agent only.
    - Both failed in early iterations: Likely a temporary issue, retry both.
    - Multiple failures: Consider if we should accept partial results or end the workflow.
    - Both successful: Mission accomplished! Time to end the workflow.

    **Remember**: You're optimizing for getting useful meeting analysis (summary + insights), not just perfect success rates.

    Use the ReAct process: THOUGHT → OBSERVATION → ACTION with clear reasoning.
    """

    print("state:", state)
    
     # Remove the action_items_count line completely
    formatted_prompt = system_prompt.format(
        summary_status=state['summary_status'],
        summary_id=f"'{state['summary_id']}'" if state.get('summary_id') else "None",
        insights_status=state['insights_status'],
        insights_id=f"'{state['insights_id']}'" if state.get('insights_id') else "None",
        iteration=state['iteration'],
        error_msg=state.get('error_message') or "None"
    )
        
    
    # Setup Groq API with Qwen model
    groq_api_key = os.getenv("GROQ_API_KEY")
    if not groq_api_key:
        raise ValueError("GROQ_API_KEY environment variable is not set")
    
    os.environ["GROQ_API_KEY"] = groq_api_key
    
    llm = ChatGroq(
        temperature=0,
        model="deepseek-r1-distill-llama-70b"  
    )
    
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Use ReAct methodology: THOUGHT → OBSERVATION → ACTION. Analyze the workflow state and decide what to do next for our goal. Think step by step.")
    ]
    
    # Get structured decision from LLM
    try:
        decision = llm.with_structured_output(SupervisorDecision).invoke(messages)
    except Exception as e:
        print(f"⚠️ LLM call failed: {e}")
        # Fallback to simulation Simulate intelligent decision  if LLM fails
        decision = simulate_smart_decision(state)
    
    print(f"💭 AI Reasoning: {decision.reasoning}")
    print(f"⚡ Decision: {decision.next_action} (confidence: {decision.confidence:.2f})")
        
    # goto = decision.next_action if decision.next_action != "end_workflow" else END

    # print('goto:',goto)

    return {
        "current_reasoning": decision.reasoning,
        "next": decision.next_action 
    }
        

In [10]:
def simulate_smart_decision(state: GraphState) -> SupervisorDecision:
    """
    Simulate what an intelligent model like Qwen would decide
    This is just simulation - replace with actual LLM call
    """
    
    summary_status = state['summary_status']
    insights_status = state['insights_status']
    iteration = state['iteration']
    
    # Smart decision making that a good reasoning model would do
    
    # Goal achieved - both successful
    if summary_status == "success" and insights_status == "success":
        return SupervisorDecision(
            next_action="end_workflow",
            reasoning="Perfect! Both summary and insights are ready. Our workflow goal is complete - we have meeting summary + key insights as requested.",
            confidence=1.0
        )
    
    # First iteration - start efficiently  
    if iteration == 1 and summary_status == "pending" and insights_status == "pending":
        return SupervisorDecision(
            next_action="call_both_parallel", 
            reasoning="First attempt - running both summary and insights agents in parallel for efficiency. This is the optimal starting strategy.",
            confidence=0.9
        )
    
    # One succeeded, one failed - targeted retry
    if summary_status == "success" and insights_status == "failed":
        if iteration <= 3:  # Smart about retry limits
            return SupervisorDecision(
                next_action="call_insights_only",
                reasoning="Summary is ready, but insights failed. Retrying only insights agent since summary is already successful. Efficient targeted approach.",
                confidence=0.8
            )
        else:
            return SupervisorDecision(
                next_action="end_workflow",
                reasoning="Summary is ready and we've tried insights multiple times. Sometimes partial success is acceptable for meeting processing workflow.",
                confidence=0.6
            )
    
    if insights_status == "success" and summary_status == "failed":
        if iteration <= 3:
            return SupervisorDecision(
                next_action="call_summary_only", 
                reasoning="Insights are ready, but summary failed. Retrying only summary agent since insights are already successful. Focused retry approach.",
                confidence=0.8
            )
        else:
            return SupervisorDecision(
                next_action="end_workflow",
                reasoning="Insights are ready and we've tried summary multiple times. We have key insights from the meeting which provides value.",
                confidence=0.6
            )
    
    # Both failed - intelligent retry decision
    if summary_status == "failed" and insights_status == "failed":
        if iteration <= 2:
            return SupervisorDecision(
                next_action="call_both_parallel",
                reasoning="Both agents failed, but it's early in the process. Likely a temporary issue (network/API). Retrying both in parallel - efficient recovery approach.",
                confidence=0.7
            )
        elif iteration <= 4:
            return SupervisorDecision(
                next_action="call_both_parallel", 
                reasoning="Multiple failures but still within reasonable retry range. The meeting document seems valid, so this might be temporary service issues. One more parallel attempt.",
                confidence=0.5
            )
        else:
            return SupervisorDecision(
                next_action="end_workflow",
                reasoning="After multiple attempts, continuing may not be productive. This could be a deeper issue with the document format or service availability. Ending workflow.",
                confidence=0.8
            )
    
    # Default intelligent fallback
    return SupervisorDecision(
        next_action="call_both_parallel",
        reasoning="Current state requires both agents to run. Taking parallel approach for efficiency in meeting processing workflow.", 
        confidence=0.6
    )


In [11]:
def route_supervisor_decision(state: GraphState) -> str:
    """
    Routes the workflow based on the decision stored by the intelligent supervisor.
    """
    return state['next'] 

In [13]:

def run_summary_agent(state: GraphState) -> dict:
    """
    An agentic node that intelligently generates a meeting summary using the
    modern langgraph.prebuilt.create_react_agent.
    """
    print("\n🤖 Modern Agentic Summary Node Called...")

    system_prompt = """You are an expert Meeting Summarization Agent. Your sole mission is to create a concise, structured, and insightful summary from the provided meeting minutes.

    ## Your Guidelines:
    1.  **Core Content**: Focus on key decisions, major topics discussed, and final outcomes.
    2.  **Tool Use**: If you identify any specific tasks or action items, you MUST use the `extract_and_store_action_items` tool to process them.
    3.  **Final Output**: In your final answer, do not list the action items . Simply state that they were identified and processed. The final output should ONLY be the clean, Markdown-formatted summary.
    """
    try:
        groq_api_key = os.getenv("GROQ_API_KEY")
        if not groq_api_key:
            raise ValueError("GROQ_API_KEY environment variable is not set")

        llm = ChatGroq(temperature=0, model="qwen/qwen3-32b")
        tools = [extract_and_store_action_items]
        summary_agent_graph = create_react_agent(llm, tools, prompt=system_prompt)

    except Exception as e:
        print(f"❌ Error during agent setup: {e}")
        return {
            "summary_status": "failed",
            "error_message": f"Summary Agent Setup Error: {str(e)}"
        }

    # Invoke the Agent with Error Handling
    try:
        document_content = DataStorage.retrieve('document_content', state['document_content_id'])
        if not document_content:
            raise ValueError("Failed to retrieve document content from storage.")

        print("🧠 Agent is thinking and generating the summary...")

        # The input for a langgraph agent is a dictionary with a "messages" key.
        agent_input = {
            "messages": [
                ("user", f"Please generate a summary for the following meeting minutes:\n\n---\n\n{document_content}")
            ]
        }
        result = summary_agent_graph.invoke(agent_input)

        # The agent's final answer is the content of the last message in the state.
        generated_summary = result["messages"][-1].content
        print("✅ Summary Agent: Generated summary successfully.:::", generated_summary)

        action_item_id = None
        for message in result.get("messages", []):
            if isinstance(message, ToolMessage) and "Confirmation: Successfully stored" in message.content:
                match = re.search(r'ID (\S+)', message.content)
                if match:
                    action_item_id = match.group(1)
                    print(f"✅ Summary node captured Action Item ID: {action_item_id}")
                    break
        
        summary_id = DataStorage.store('summary_output', generated_summary)

        # Prepare the dictionary to return and update the state
        return_data = {
            "summary_status": "success",
            "summary_id": summary_id,
            "error_message": ""
        }
        
        # If we found an action item ID, add it to the state update
        if action_item_id:
            return_data["action_items_id"] = action_item_id
        return return_data
    
    except Exception as e:
        print(f"❌ Error in Agentic Summary Node: {e}")
        return {
            "summary_status": "failed",
            "error_message": f"Summary Agent Error: {str(e)}"
        }

In [16]:

def run_insights_agent(state: GraphState) -> dict:
    """
    A specialist node that uses a focused LLM chain to extract deep,
    strategic insights from meeting minutes.
    """
    print("\n💡 Specialist Insights Node Called...")

    # 1. Crafting a High-Fidelity Prompt for the Insights Specialist
    # This is the heart of the "agentic" behavior for a no-tool task.
    # We give it a strong persona, a clear mission, and a structured framework.
    prompt_template = ChatPromptTemplate.from_template(
        """
        # PERSONA & MISSION
        You are a premier Business Strategy Analyst and Insights Specialist. You are not a summarizer; you are a sense-maker. Your mission is to transcend the surface-level details of the provided meeting minutes and distill them into high-level, actionable, strategic insights for executive review. You must uncover the 'why' behind the 'what'.

        # ANALYTICAL FRAMEWORK
        Read the entire document first. Then, apply the following framework to generate your insights. Do not mention this framework in your output; use it as your internal guide.

        1.  **Identify Key Themes**: What are the recurring strategic ideas, concerns, or opportunities being discussed? Look for patterns, not just topics.
            - *Example: "A recurring theme was the tension between innovation speed and maintaining product quality."*

        2.  **Analyze Critical Decisions & Implications**: For each major decision, state it concisely and then, most importantly, explain its strategic implication.
            - *Example: "Decision: The 'Phoenix Project' was greenlit. Implication: This signals a major strategic pivot for the company, deprioritizing legacy systems to capture a new market segment."*

        3.  **Surface Actionable Insights**: What can the leadership team learn from the conversation? These are not action items (tasks for individuals), but strategic recommendations for the team or company as a whole.
            - *Example: "Insight: The extended debate over resource allocation for Q4 reveals a potential misalignment on departmental priorities. A cross-departmental priority-setting workshop is recommended."*

        # OUTPUT REQUIREMENTS
        - Your final output must be ONLY the structured insights.
        - Use clean, professional Markdown formatting (headings, subheadings, bullet points).
        - Do NOT summarize the meeting. Do NOT mention action items. Your focus is exclusively on strategic insights.
        - Begin your analysis directly without any preamble like "Here are the insights...".

        # DOCUMENT FOR ANALYSIS
        ---
        {document_content}
        ---
                """
            )

    try:
        groq_api_key = os.getenv("GROQ_API_KEY")
        if not groq_api_key:
            raise ValueError("GROQ_API_KEY environment variable is not set")

        # A powerful model is essential for this kind of deep reasoning task.
        llm = ChatGroq(temperature=0.2, model="llama3-70b-8192")

        # This simple LCEL chain is more efficient than an agent for no-tool tasks.
        insights_chain = prompt_template | llm | StrOutputParser()

    except Exception as e:
        print(f"❌ Error during insights chain setup: {e}")
        return {
            "insights_status": "failed",
            "error_message": f"Insights Chain Setup Error: {str(e)}"
        }

    try:
        document_content = DataStorage.retrieve('document_content', state['document_content_id'])
        if not document_content:
            raise ValueError("Failed to retrieve document content from storage.")

        print("🧠 Specialist is analyzing and extracting insights...")
        generated_insights = insights_chain.invoke({"document_content": document_content})
        print("✅ Insights Specialist: Extracted insights successfully.", generated_insights)

        insights_id = DataStorage.store('insights_output', generated_insights)

        return {
            "insights_status": "success",
            "insights_id": insights_id,
            "error_message": ""
        }

    except Exception as e:
        print(f"❌ Error in Specialist Insights Node: {e}")
        return {
            "insights_status": "failed",
            "error_message": f"Insights Specialist Error: {str(e)}"
        }

In [17]:
def run_both_parallel_agents(state: GraphState) -> dict:
    """Runs the refactored summary and insights agents in parallel."""
    print("\n---RUNNING BOTH REFACTORED AGENTS IN PARALLEL---")

    parallel_runnable = RunnableParallel(
        summary_result=run_summary_agent,
        insights_result=run_insights_agent
    )
    parallel_results = parallel_runnable.invoke(state)
    
    return {**parallel_results['summary_result'], **parallel_results['insights_result']}

In [18]:
def increment_iteration(state: GraphState) -> GraphState:
    """Track iterations for intelligent decision making"""
    new_state = state.copy()
    new_state['iteration'] += 1
    print(f"\n--- Workflow Iteration: {new_state['iteration']} ---")
    return new_state

In [19]:
# MCP Integration via Singleton Manager
import time
import json
import threading
from queue import Queue, Empty
import os

class MCPManager:
    """Singleton MCP Manager for notebook integration with external MCP server"""
    _instance = None
    _lock = threading.Lock()
    
    def __new__(cls):
        if cls._instance is None:
            with cls._lock:
                if cls._instance is None:
                    cls._instance = super().__new__(cls)
                    cls._instance._initialized = False
        return cls._instance
    
    def __init__(self):
        if not self._initialized:
            self.mcp_client = None
            self.debug = True
            self._initialized = True
    
    def _get_mcp_client(self):
        """Lazy initialization of MCP client"""
        if self.mcp_client is None:
            try:
                # Import the MCP client from your separate file
                os.chdir('../')
                print(os.getcwd)
                from src.mcpserver import MCPClient  
                self.mcp_client = MCPClient()
                print("📦 MCP Client imported successfully")
            except ImportError as e:
                print(f"❌ Failed to import MCPClient: {e}")
                print("   Make sure mcpserver.py is in your Python path")
                return None
        return self.mcp_client
    
    def start_server(self):
        """Start MCP server and initialize connection"""
        client = self._get_mcp_client()
        if not client:
            return False
        
        try:
            # Start the server process
            if not client.start_server():
                print("❌ Failed to start MCP server")
                return False
            
            # Initialize the connection
            if not client.initialize():
                print("❌ Failed to initialize MCP connection")
                return False
            
            print("✅ MCP server started and initialized successfully")
            return True
            
        except Exception as e:
            print(f"❌ Error starting MCP server: {e}")
            return False
    
    def is_server_running(self):
        """Check if MCP server is running"""
        client = self._get_mcp_client()
        if not client:
            return False
        return hasattr(client, 'running') and client.running and client.initialized
    
    def list_tools(self):
        """List available MCP tools"""
        if not self.is_server_running():
            print("⚠️  Server not running, starting...")
            if not self.start_server():
                return {"error": "Failed to start MCP server"}
        
        client = self._get_mcp_client()
        if not client:
            return {"error": "MCP client not available"}
        
        try:
            result = client.list_tools()
            if self.debug:
                print(f"[TOOLS LIST] {json.dumps(result, indent=2)}")
            return result
        except Exception as e:
            return {"error": f"Failed to list tools: {str(e)}"}
    
    def call_tool(self, tool_name: str, arguments: dict = None) -> dict:
        """Call a specific MCP tool"""
        if not self.is_server_running():
            print("⚠️  Server not running, starting...")
            if not self.start_server():
                return {"error": "Failed to start MCP server"}
        
        client = self._get_mcp_client()
        if not client:
            return {"error": "MCP client not available"}
        
        try:
            result = client.call_tool(tool_name, arguments or {})
            if self.debug:
                print(f"[TOOL CALL] {tool_name} -> {json.dumps(result, indent=2)}")
            return result
        except Exception as e:
            return {"error": f"Failed to call tool {tool_name}: {str(e)}"}
    
    def mcp_request(self, tool_name: str, args: dict = None) -> dict:
        """Alias for call_tool to match your original interface"""
        return self.call_tool(tool_name, args)
    
    def get_server_status(self):
        """Get comprehensive server status"""
        client = self._get_mcp_client()
        if not client:
            return {"status": "client_not_available", "running": False}
        
        status = {
            "running": self.is_server_running(),
            "client_available": True,
            "process_id": getattr(client.process, 'pid', None) if hasattr(client, 'process') and client.process else None
        }
        
        if status["running"]:
            # Try to get tools to verify connectivity
            try:
                tools_response = self.list_tools()
                if (
                    "result" in tools_response
                    and isinstance(tools_response["result"], dict)
                    and "tools" in tools_response["result"]
                    and isinstance(tools_response["result"]["tools"], list)
                ):
                    status["tools_count"] = len(tools_response["result"]["tools"])
                    status["status"] = "healthy"
                    status["tools"] = [t["name"] for t in tools_response["result"]["tools"]]
                else:
                    status["status"] = "connection_issues"
                    status["error"] = tools_response.get("error", "Unknown error")
            except Exception as e:
                status["status"] = "connection_error"
                status["error"] = str(e)
        else:
            status["status"] = "not_running"
        
        return status
    
    def restart_server(self):
        """Restart the MCP server"""
        print("🔄 Restarting MCP server...")
        self.stop_server()
        time.sleep(2)
        return self.start_server()
    
    def stop_server(self):
        """Stop the MCP server"""
        client = self._get_mcp_client()
        if client and hasattr(client, 'stop'):
            try:
                client.stop()
                print("✅ MCP server stopped")
            except Exception as e:
                print(f"⚠️  Error stopping server: {e}")
    
    def test_connection(self):
        """Test MCP connection with basic tool listing"""
        print("🧪 Testing MCP connection...")
        
        status = self.get_server_status()
        print(f"📊 Server Status: {status['status']}")
        
        if status["status"] == "healthy":
            tools = status.get("tools", [])
            print(f"✅ Connection successful! Found {len(tools)} tools:")
            for i, tool_name in enumerate(tools[:5], 1):  # Show first 5 tools
                print(f"  {i}. {tool_name}")
            if len(tools) > 5:
                print(f"  ... and {len(tools) - 5} more tools")
            return True
        else:
            print(f"❌ Connection failed: {status.get('error', 'Unknown error')}")
            return False
    
    def show_tools(self):
        """Display available tools in a user-friendly format"""
        tools_response = self.list_tools()
        
        if "error" in tools_response:
            print(f"❌ Error getting tools: {tools_response['error']}")
            return
        
        if "result" not in tools_response:
            print("❌ No tools found or invalid response")
            return

        result = tools_response["result"]
        # Handle both dict and list cases for result
        if isinstance(result, dict) and "tools" in result and isinstance(result["tools"], list):
            tools = result["tools"]
        elif isinstance(result, list):
            tools = result
        else:
            print("❌ No tools found or invalid response structure")
            return

        print(f"\n📋 Available Tools ({len(tools)}):")
        print("=" * 50)
        
        for i, tool in enumerate(tools, 1):
            print(f"{i}. 🔧 {tool.get('name', 'Unnamed Tool')}")
            if 'description' in tool:
                print(f"   📝 {tool['description']}")
            if 'inputSchema' in tool and isinstance(tool['inputSchema'], dict) and 'properties' in tool['inputSchema']:
                props = tool['inputSchema']['properties']
                if props:
                    print(f"   📥 Parameters: {', '.join(props.keys())}")
            print()
    
    def quick_call(self, tool_name: str, **kwargs):
        """Quick tool call with keyword arguments"""
        return self.call_tool(tool_name, kwargs)

# Initialize MCP manager (singleton)
print("🔧 Initializing MCP Manager...")
mcp_manager = MCPManager()

# Test if we can import the MCP client
try:
    client_test = mcp_manager._get_mcp_client()
    if client_test:
        print("✅ MCP Client import successful")
        
        # Auto-start server
        print("🚀 Starting MCP server...")
        if mcp_manager.start_server():
            print("🎉 MCP Manager ready for use!")
            
            # Quick connection test
            if mcp_manager.test_connection():
                print("\n📋 Use mcp_manager.show_tools() to see all available tools")
        else:
            print("⚠️  MCP server failed to start. Use mcp_manager.start_server() to retry")
    else:
        print("⚠️  MCP Client not available. Make sure mcpserver.py is accessible")
        
except Exception as e:
    print(f"⚠️  MCP initialization error: {e}")
    print("   The manager is available but server needs manual start")


🔧 Initializing MCP Manager...
<built-in function getcwd>
📦 MCP Client imported successfully
✅ MCP Client import successful
🚀 Starting MCP server...
⏳ Waiting for MCP server to start...
[MCP STDERR] Starting Slack MCP Server...
[MCP STDERR] Connecting server to transport...
[MCP STDERR] Slack MCP Server running on stdio
✅ MCP Server started successfully

🤝 Sending initialize...
[MCP REQUEST] {
  "jsonrpc": "2.0",
  "method": "initialize",
  "params": {
    "protocolVersion": "2024-11-05",
    "capabilities": {},
    "clientInfo": {
      "name": "mcp-python-client",
      "version": "1.0"
    }
  },
  "id": 1
}
[MCP STDOUT] {"result":{"protocolVersion":"2024-11-05","capabilities":{"tools":{}},"serverInfo":{"name":"Slack MCP Server","version":"1.0.0"}},"jsonrpc":"2.0","id":1}
[MCP RESPONSE] {
  "result": {
    "protocolVersion": "2024-11-05",
    "capabilities": {
      "tools": {}
    },
    "serverInfo": {
      "name": "Slack MCP Server",
      "version": "1.0.0"
    }
  },
  "jsonrpc

[MCP STDERR] Received CallToolRequest: {
[MCP STDERR] method: 'tools/call',
[MCP STDERR] params: {
[MCP STDERR] name: 'slack_post_message',
[MCP STDERR] arguments: {
[MCP STDERR] channel_id: 'general',
[MCP STDERR] text: '**# Meeting Summary - July 22, 2025**\n' +
[MCP STDERR] '**Attendees**: Alice, Bob, Charlie\n' +
[MCP STDERR] '\n' +
[MCP STDERR] '### 1. Project Phoenix\n' +
[MCP STDERR] '- **Status Update**: Server deployment confirmed by Bob.  \n' +
[MCP STDERR] '- **Next Steps**: Action item identified and processed for budget review scheduling.\n' +
[MCP STDERR] '\n' +
[MCP STDERR] '### 2. Marketing\n' +
[MCP STDERR] "- **Announcement**: 'Summer Sale' launch set for August 1st.  \n" +
[MCP STDERR] '- **Next Steps**: Action item identified and processed for BERT chatbot refinement.\n' +
[MCP STDERR] '\n' +
[MCP STDERR] 'All action items have been extracted and stored for tracking.\n' +
[MCP STDERR] '\n' +
[MCP STDERR] '\n' +
[MCP STDERR] '    **Key Themes**\n' +
[MCP STDERR] '===

In [40]:
mcp_manager.call_tool(tool_name= "slack_post_message", arguments= {
    "channel_id": 'C0934AS6UB1', 
    "text": "testing at the time of merging the first and second version of the notebook"
})

[MCP REQUEST] {
  "jsonrpc": "2.0",
  "method": "tools/call",
  "params": {
    "name": "slack_post_message",
    "arguments": {
      "channel_id": "C0934AS6UB1",
      "text": "testing at the time of merging the first and second version of the notebook"
    }
  },
  "id": 3
}
[MCP RESPONSE] {
  "result": {
    "content": [
      {
        "type": "text",
        "text": "{\"ok\":true,\"channel\":\"C0934AS6UB1\",\"ts\":\"1753434481.005639\",\"message\":{\"user\":\"U0946J1TQHW\",\"type\":\"message\",\"ts\":\"1753434481.005639\",\"bot_id\":\"B0946J1T4M6\",\"app_id\":\"A093GGX8TE1\",\"text\":\"testing at the time of merging the first and second version of the notebook\",\"team\":\"T093JHQS18A\",\"bot_profile\":{\"id\":\"B0946J1T4M6\",\"app_id\":\"A093GGX8TE1\",\"user_id\":\"U0946J1TQHW\",\"name\":\"meetingplus\",\"icons\":{\"image_36\":\"https://a.slack-edge.com/80588/img/plugins/app/bot_36.png\",\"image_48\":\"https://a.slack-edge.com/80588/img/plugins/app/bot_48.png\",\"image_72\":\"ht

{'result': {'content': [{'type': 'text',
    'text': '{"ok":true,"channel":"C0934AS6UB1","ts":"1753434481.005639","message":{"user":"U0946J1TQHW","type":"message","ts":"1753434481.005639","bot_id":"B0946J1T4M6","app_id":"A093GGX8TE1","text":"testing at the time of merging the first and second version of the notebook","team":"T093JHQS18A","bot_profile":{"id":"B0946J1T4M6","app_id":"A093GGX8TE1","user_id":"U0946J1TQHW","name":"meetingplus","icons":{"image_36":"https://a.slack-edge.com/80588/img/plugins/app/bot_36.png","image_48":"https://a.slack-edge.com/80588/img/plugins/app/bot_48.png","image_72":"https://a.slack-edge.com/80588/img/plugins/app/service_72.png"},"deleted":false,"updated":1751170336,"team_id":"T093JHQS18A"},"blocks":[{"type":"rich_text","block_id":"mdj","elements":[{"type":"rich_text_section","elements":[{"type":"text","text":"testing at the time of merging the first and second version of the notebook"}]}]}]},"warning":"missing_charset","response_metadata":{"warnings":["m

In [20]:
#  Enhanced Tools with MCP Integration via Stdio
@tool
def slack_post_message(channel_id: str, text: str) -> dict:
    """Post message to Slack channel via MCP server"""
    # Truncate to Slack's limits
    if len(text) > 3900:
        text = text[:3900] + "\n\n[Message truncated]"
    
    return mcp_manager.call_tool(
       tool_name = "slack_post_message",
        arguments = {"channel_id": channel_id, "text": text}
    )

@tool
def slack_list_channels(limit: int = 100, cursor: Optional[str] = None) -> dict:
    """List accessible Slack channels via MCP server"""
    args = {"limit": limit}
    if cursor:
        args["cursor"] = cursor
    return mcp_manager.call_tool(tool_name = "slack_list_channels", arguments = args)

@tool
def slack_get_users(limit: int = 100, cursor: Optional[str] = None) -> dict:
    """Get workspace users via MCP server"""
    args = {"limit": limit}
    if cursor:
        args["cursor"] = cursor
    return mcp_manager.call_tool( tool_name ="slack_get_users", arguments= args)

@tool
def slack_find_user_by_name(name: str) -> Optional[str]:
    """Find user by display name using MCP tools"""
    try:
        result = slack_get_users(limit=200)
        result1 = result.get("result", [])

        res2 = result1.get('content')[0].get('text')

        raw = json.loads(res2)

        users = raw.get('members')
        
        for user in users:

            print(user)
            
            if name.lower() in [
                    user.get("name", "").lower(),
                    user.get("real_name", "").lower(),
                    user.get("profile", "").get('real_name').lower()
             ]:
                return { "user_id" : user["id"], "user_name":user }
        return None
    except Exception:
        return None

In [51]:
def slack_node( message : str ):
 
    print("*****************slack NODE CALLED************")
    
   
        # The final, most robust version of the prompt
    system_prompt = """
    # PERSONA
    You are a methodical and precise Slack Workflow Automation Agent. Your function is to communicate important information to the correct audience.

    # OVERALL STRATEGY
    You will operate in two distinct phases. You MUST complete Phase 1 entirely before beginning Phase 2.

    ---
    # PHASE 1: General Announcement
    Your goal in this phase is to post the main meeting summary and insights to the primary company channel.

    1.  **Find Main Channel**: Use `slack_list_channels` to find the single most appropriate public channel for company-wide announcements. Look for names like 'general', 'announcements', or 'all-abc'. Remember this channel's ID.
    2.  **Post Main Content**: Use `slack_post_message` to post the formatted "Final Summary" and "Final Insights" to the main channel ID you just found.

    ---
    # PHASE 2: Action Item Checklist
    After Phase 1 is complete, you will work through a checklist of ALL action items provided. You are NOT allowed to provide your final answer until every single item from the list has been processed.

    3.  **Iterate Through Checklist**: For each action item, perform the following steps:
        a. **Determine Target Channel**:
            - First, check if the action item `owner` (e.g., "AI team") exactly matches a channel name you found earlier (e.g., 'ai-team').
            - If it's a direct match, use that specific channel's ID.
            - If there is no matching channel, default to using the main channel ID from Phase 1.
        b. **Find User to Mention**: Use `slack_find_user_by_name` to get the user ID for the action item's `owner`.
        c. **Format Mention**:
            - If a user ID is found, format it for Slack mentioning (e.g., `<@USER_ID>`).
            - If no user is found or the owner is a team name, format the name in bold (e.g., `*AI team*`).
        d. **Post Notification**: Use `slack_post_message` to send the notification for this single action item to the target channel determined in step 3a.
        e. **Review and Repeat**: After posting, mentally check off the item. If there are more action items left in the initial list, you MUST immediately repeat steps 3a-3d for the next item. DO NOT stop and give a final answer yet.

    # CRITICAL RULES
    - **Complete the Checklist**: You must process ALL action items before providing a final summary of your work.
    - **Tool Parameter Integrity**: `channel_id` MUST be a channel ID (starts with 'C').
    - **Error Handling**: If a user is not found, format their name in bold and proceed. Do not stop.
    """
        
    # system_prompt = """
    # # PERSONA
    # You are a methodical and precise Slack Workflow Automation Agent. Your function is to communicate important information to the correct audience.

    # # OVERALL STRATEGY
    # You will operate in two distinct phases. You MUST complete Phase 1 entirely before beginning Phase 2.

    # ---
    # # PHASE 1: General Announcement
    # Your goal in this phase is to post the main meeting summary and insights to the primary company channel.

    # 1.  **Find Main Channel**: Use the `slack_list_channels` tool to find the single most appropriate public channel for company-wide announcements. Look for names like 'general', 'announcements', or 'all-abc'.
    # 2.  **Post Main Content**: Use `slack_post_message` to post the formatted "Final Summary" and "Final Insights" to the channel ID you discovered in the previous step.

    # ---
    # # PHASE 2: Action Item Notification
    # After Phase 1 is complete, your goal is to notify the owners of their specific action items.

    # 3.  **Process EACH Action Item**: For every action item in the provided list, perform the following sub-steps:
    #     a. **Find User**: Use `slack_find_user_by_name` to find the user ID for the action item's `owner`.
    #     b. **Format Mention**:
    #         - If a user ID is found, format it for Slack mentioning (e.g., `<@USER_ID>`).
    #         - If no user is found or the owner is a team, format the name in bold (e.g., `*AI team*`).
    #     c. **Post Notification**: Use `slack_post_message` to send a message containing the specific `Task` and `Deadline` to the **main channel** you found in Phase 1. Include the formatted mention from the previous sub-step.

    # # CRITICAL RULES
    # - **Phased Execution**: You MUST successfully complete Phase 1 before starting Phase 2.
    # - **Tool Parameter Integrity**: Pay close attention to tool parameters. The `channel_id` for `slack_post_message` MUST be a channel ID (starts with 'C'), never a user ID (starts with 'U').
    # - **Error Handling**: If a tool fails (e.g., `slack_find_user_by_name` returns null), do not get stuck. Report the failure clearly for that specific action item in your final answer and move on to the next one.
    # """
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
    ])

    groq_api_key = os.getenv("GROQ_API_KEY")
    if not groq_api_key:
        raise ValueError("GROQ_API_KEY environment variable is not set")

    # llm = ChatGroq(temperature=0, model="qwen/qwen3-32b")
    llm = ChatGroq(
        temperature=0.2,
        model="gemma2-9b-it"  
    )
    # llama-3.3-70b-versatile, llama3-70b-8192 

    # Create the react agent with tools
    slack_agent = create_react_agent(
        model=llm,
        tools=[slack_find_user_by_name, slack_get_users, slack_post_message, slack_list_channels ],
        prompt=prompt
    )
    
    try:
        
        result = slack_agent.invoke({"messages": message})
        
        print(f"Slack agent result: {result}")

        return result
    
    except Exception as e:
        print(f" Error in slack_node: {e}")


In [52]:
result = slack_node( """Final Summary:
 # Meeting Summary - July 23, 2025  
**Attendees**: Alice, Bob, Charlie  

### 1. Project Phoenix  
- **Status Update**: Bob confirmed successful server deployment.  
- **Next Steps**: Action item identified for budget review (processed via tool).  

### 2. Marketing  
- **Summer Sale**: Launch scheduled for August 1st.  
- **Technical Prep**: Action item assigned to refine BERT chatbot for user intent analysis (processed via tool).  

All action items have been extracted and stored for tracking.

✅ Final Insights:
 **Key Themes**
===============

* **Strategic Resource Allocation**: The meeting minutes hint at potential misalignment on departmental priorities, with discussions around budget reviews and resource allocation for Project Phoenix and the Summer Sale.

**Critical Decisions & Implications**
=====================================

* **Project Phoenix Deployment**: Decision: The server has been deployed. Implication: This marks a significant milestone in the project's timeline, likely to impact the company's product offerings and market positioning.
* **Summer Sale Launch**: Decision: The sale will launch on August 1st. Implication: This strategic move aims to drive revenue growth during a critical period, potentially influencing customer behavior and loyalty.

**Actionable Insights**
=====================

* **Prioritization Alignment**: The leadership team should consider a cross-departmental priority-setting workshop to ensure alignment on resource allocation and strategic objectives.
* **Strategic Initiative Interdependencies**: The company should assess how Project Phoenix and the Summer Sale intersect and impact each other, to maximize synergies and minimize potential conflicts.

✅ Extracted Action Items:
 retrievel data of type 'action_items' with ID: action_items_1753415912936_875
 Task : schedule a budget review, 
 	 owner: Charlie 	 deadline: Friday 
 Task : refine the BERT chatbot for user intent, 
 	 owner: AI team 	 deadline: July 28th """)

*****************slack NODE CALLED************
[MCP REQUEST] {
  "jsonrpc": "2.0",
  "method": "tools/call",
  "params": {
    "name": "slack_list_channels",
    "arguments": {
      "limit": 100
    }
  },
  "id": 8
}
[MCP RESPONSE] {
  "result": {
    "content": [
      {
        "type": "text",
        "text": "{\"ok\":true,\"channels\":[{\"id\":\"C093KK8DBGC\",\"name\":\"ai-team\",\"is_channel\":true,\"is_group\":false,\"is_im\":false,\"is_mpim\":false,\"is_private\":false,\"created\":1751170632,\"is_archived\":false,\"is_general\":false,\"unlinked\":0,\"name_normalized\":\"ai-team\",\"is_shared\":false,\"is_org_shared\":false,\"is_pending_ext_shared\":false,\"pending_shared\":[],\"context_team_id\":\"T093JHQS18A\",\"updated\":1751170632496,\"parent_conversation\":null,\"creator\":\"U093JHQT2JW\",\"is_ext_shared\":false,\"shared_team_ids\":[\"T093JHQS18A\"],\"pending_connected_team_ids\":[],\"is_member\":true,\"last_read\":\"1751276461.859299\",\"topic\":{\"value\":\"\",\"creator\"

In [53]:
if result and "messages" in result:
    for message in result["messages"]:
        if hasattr(message, "pretty_print"):
            message.pretty_print()
        else:
            print(message)
else:
    print("No messages found in result.")

================================ Human Message =================================

Final Summary:
 # Meeting Summary - July 23, 2025  
**Attendees**: Alice, Bob, Charlie  

### 1. Project Phoenix  
- **Status Update**: Bob confirmed successful server deployment.  
- **Next Steps**: Action item identified for budget review (processed via tool).  

### 2. Marketing  
- **Summer Sale**: Launch scheduled for August 1st.  
- **Technical Prep**: Action item assigned to refine BERT chatbot for user intent analysis (processed via tool).  

All action items have been extracted and stored for tracking.

✅ Final Insights:
 **Key Themes**

* **Strategic Resource Allocation**: The meeting minutes hint at potential misalignment on departmental priorities, with discussions around budget reviews and resource allocation for Project Phoenix and the Summer Sale.

**Critical Decisions & Implications**

* **Project Phoenix Deployment**: Decision: The server has been deployed. Implication: This marks a signi

In [21]:
State = {'document_content_id': 'document_content_1753415875925_299', 'summary_status': 'success', 'insights_status': 'success', 'summary_id': 'summary_output_1753415881876_662', 'insights_id': 'insights_output_1753415879390_421', 'action_items_id': 'action_items_1753415880811_634', 'iteration': 1, 'error_message': '', 'current_reasoning': 'Both summary and insights tasks have successfully completed with their IDs present. The workflow goal has been achieved first layer , so moving  the workflow is the appropriate next step.', 'next': 'run_slack_orchestrator_node', 'slack_tasks_completed': []}


In [ ]:
def run_summary_poster_specialist(state: Any) -> str:
    """
    A specialist agent that finds the main channel and posts the summary/insights.
    """
    print("--- specialist: SummaryPoster running ---")
    summary = DataStorage.retrieve('summary_output', state['summary_id'])
    insights = DataStorage.retrieve('insights_output', state['insights_id'])
    if not summary or not insights:
        print("❌ Summary or insights not found in storage.")
        # return "ERROR: Summary or insights data is missing."
    
    summary = """
     # Meeting Summary - July 22, 2025  
**Attendees**: Alice, Bob, Charlie  

### 1. Project Phoenix  
- **Status Update**: Server deployment confirmed by Bob.  
- **Next Steps**: Action item identified and processed for budget review scheduling.  

### 2. Marketing  
- **Announcement**: 'Summer Sale' launch set for August 1st.  
- **Next Steps**: Action item identified and processed for BERT chatbot refinement.  

All action items have been extracted and stored for tracking.

    """

    insights = """
    **Key Themes**
===============

* **Strategic Prioritization**: The meeting minutes suggest a focus on allocating resources to high-priority projects, such as Project Phoenix, and optimizing marketing efforts for the Summer Sale.

**Critical Decisions & Implications**
=====================================

* **Project Phoenix Deployment**: Decision: The server for Project Phoenix has been deployed. Implication: This marks a significant milestone in the project's development, indicating a commitment to investing in innovative initiatives.
* **Summer Sale Launch**: Decision: The Summer Sale will launch on August 1st. Implication: This strategic marketing move aims to drive revenue growth during a critical period, highlighting the company's focus on short-term results.

"""
    content_to_post = summary + "\n\n" + insights
    # This agent gets a very simple, focused prompt
    system_prompt ="""
    # MISSION
Your sole mission is to post a provided message to the single most appropriate public announcement channel in Slack.

# PROCEDURE
You MUST follow these steps in order:
1.  Use the `slack_list_channels` tool to see a list of available public channels.
2.  From that list, identify the single best channel for a company-wide announcement. Look for names like 'all-abc' or 'general'. You MUST use the channel's real ID from the tool's output.
3.  Use the `slack_post_message` tool to send the user's content to the channel ID you identified in Step 2.

# RULES
- Do not invent channel names. You must use `slack_list_channels` first.
- Your final answer should be a simple confirmation message.
    """

    groq_api_key = os.getenv("GROQ_API_KEY")
    if not groq_api_key:
        raise ValueError("GROQ_API_KEY environment variable is not set")

    # llm = ChatGroq(temperature=0, model="qwen/qwen3-32b")
    llm = ChatGroq(
        temperature=0.2,
        model="gemma2-9b-it"  
    )
    
    agent = create_react_agent(
        model=llm, 
        tools=[slack_list_channels, slack_post_message],
        prompt=ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="messages"),
        ])
    )
    
    result = agent.invoke({
        "messages": [("user", content_to_post)]
    })
    print(result)
    
    # Returns a simple status message to the orchestrator
    return "SUCCESS: Summary and insights have been posted."

In [34]:
def run_action_item_router_specialist(state: Any) -> str:
    """
    A specialist agent that iterates through action items and notifies owners.
    """
    print("--- specialist: ActionItemRouter running ---")
    action_items_list = DataStorage.retrieve('action_items', state['action_items_id'])
    
    if not action_items_list:
        print("❌ Action items list not found in storage.")
        # return "ERROR: Action items data is missing."

    action_items_list = """
     - Retrieved 2 action items.
--- Action Items Details ---
- Task: schedule a budget review, 
 	 Owner: Charlie,
 	 Deadline: Friday
- Task: refine the BERT chatbot for user intent, 
 	 Owner: AI team,
 	 Deadline: July 28th
      ================

* **Resource Allocation**: Insight: The emphasis on Project Phoenix and the Summer Sale implies a potential need for more deliberate resource allocation and prioritization across departments to ensure alignment with company goals.
* **Innovation vs. Operational Efficiency**: Insight: The deployment of Project Phoenix and the refinement of the BERT chatbot suggest a tension between investing in innovative projects and optimizing existing operations, highlighting the need for a balanced approach to drive long-term success.

     """

    # We use the powerful, looping "Checklist" prompt here
    system_prompt = """
    # You are an Action Item Notification Bot.
    # Your only job is to process the following list of action items and notify the owners in Slack.
    # You must follow the checklist procedure and process every single item.
    # ... (The full, detailed Phase 2 prompt from our previous discussion) ...
    """
    groq_api_key = os.getenv("GROQ_API_KEY")
    if not groq_api_key:
        raise ValueError("GROQ_API_KEY environment variable is not set")

    # llm = ChatGroq(temperature=0, model="qwen/qwen3-32b")
    llm = ChatGroq(
        temperature=0.2,
        model="gemma2-9b-it"  
    )
    
    agent = create_react_agent(
        model=llm, 
        tools=[slack_list_channels, slack_find_user_by_name, slack_post_message],
        prompt=ChatPromptTemplate.from_messages([("system", system_prompt), MessagesPlaceholder(variable_name="messages")])
    )

    initial_message = f"Please process this list of action items: {action_items_list}"
    result = agent.invoke({"messages": [("user", initial_message)]})
    
    return f"SUCCESS: Action item processing complete. Final status: {result['messages'][-1].content}"

In [44]:
# These are the "tools" for the Orchestrator agent.
# It doesn't call Slack tools directly; it calls other agents.

# @tool
def post_summary_and_insights(state: Any) -> str:
    """
    Call this tool first to post the main meeting summary and insights to a public channel.
    """
    return run_summary_poster_specialist(state)

# @tool
def notify_action_item_owners(state: Any) -> str:
    """
    Call this tool AFTER the summary has been posted to process all action items and notify their owners.
    """
    return run_action_item_router_specialist(state)

In [45]:
def run_slack_orchestrator_node(state: Any) -> dict:
    """
    The main "manager" agent. It analyzes the state and delegates tasks
    to specialist agents.
    """
    print("\nOrchestrator Agent Called...")

    # The Orchestrator gets a high-level strategic prompt
    system_prompt = """
    You are an Orchestrator Agent. Your job is to manage the process of distributing meeting results to Slack.
    Analyze the `slack_tasks_completed` list in the current state to decide what to do next.

    1. Your first job is to post the summary. If 'summary_posted' is NOT in `slack_tasks_completed`, you MUST call the `post_summary_and_insights` tool.
    2. After the summary is posted, your second job is to handle action items. If 'summary_posted' IS in `slack_tasks_completed` but 'action_items_routed' is NOT, you MUST call the `notify_action_item_owners` tool.
    3. Once both tasks are done, you can finish.
    """

    groq_api_key = os.getenv("GROQ_API_KEY")
    if not groq_api_key:
        raise ValueError("GROQ_API_KEY environment variable is not set")

    # llm = ChatGroq(temperature=0, model="qwen/qwen3-32b")
    llm = ChatGroq(
        temperature=0.2,
        model="gemma2-9b-it"  
    )

    orchestrator_agent = create_react_agent(
        model=llm,
        tools=[post_summary_and_insights, notify_action_item_owners],
        prompt=ChatPromptTemplate.from_messages([("system", system_prompt), MessagesPlaceholder(variable_name="messages")])
    )

    # We need to format the state into a message for the agent to read
    # This is how the agent gets its context
    state_summary = f"Current state: slack_tasks_completed = {state.get('slack_tasks_completed', [])}"
    
    # We invoke the orchestrator in a loop to handle the sequence
    # Note: In a real LangGraph setup, the graph's own loop would handle this.
    # This is a simplified representation.
    
    if "summary_posted" not in state.get('slack_tasks_completed', []):
        print("Orchestrator Decision: Calling Summary specialist...")
        # In a real graph, we'd wrap the tool call to pass the state implicitly
        # For this outline, we'll simulate the agent's decision
        result_summary = post_summary_and_insights(state)
        print(f"Orchestrator received: {result_summary}")
        state['slack_tasks_completed'].append('summary_posted')

    if "action_items_routed" not in state.get('slack_tasks_completed', []):
        print("Orchestrator Decision: Calling Action Item specialist...")
        result_action_items = notify_action_item_owners(state)
        print(f"Orchestrator received: {result_action_items}")
        state['slack_tasks_completed'].append('action_items_routed')

    return state

In [46]:
# class GraphState(TypedDict):
#     """Refactored state using IDs for scalability."""
#     document_content_id: str
#     summary_status: str
#     insights_status: str
#     summary_id: Optional[str]
#     insights_id: Optional[str]
#     action_items_id: Optional[str]
#     iteration: int
#     error_message: str
#     current_reasoning: str
#     next: str
#     slack_tasks_completed: List[str]


In [47]:
# workflow = StateGraph(GraphState)
# workflow.add_node("run_slack_orchestrator_node", run_slack_orchestrator_node)
# workflow.add_node("run_summary_agent", run_summary_agent)
# workflow.add_edge("run_slack_orchestrator_node", END)


In [48]:

result = run_slack_orchestrator_node(State)


Orchestrator Agent Called...
Orchestrator Decision: Calling Summary specialist...
--- specialist: SummaryPoster running ---
 retrievel data of type 'summary_output' with ID: summary_output_1753415881876_662
 retrievel data of type 'insights_output' with ID: insights_output_1753415879390_421
❌ Summary or insights not found in storage.
[MCP REQUEST] {
  "jsonrpc": "2.0",
  "method": "tools/call",
  "params": {
    "name": "slack_post_message",
    "arguments": {
      "channel_id": "general",
      "text": "**# Meeting Summary - July 22, 2025**\n**Attendees**: Alice, Bob, Charlie\n\n### 1. Project Phoenix\n- **Status Update**: Server deployment confirmed by Bob.  \n- **Next Steps**: Action item identified and processed for budget review scheduling.\n\n### 2. Marketing\n- **Announcement**: 'Summer Sale' launch set for August 1st.  \n- **Next Steps**: Action item identified and processed for BERT chatbot refinement.\n\nAll action items have been extracted and stored for tracking.\n\n\n   

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT

In [ ]:
if result and "messages" in result:
    for message in result["messages"]:
        if hasattr(message, "pretty_print"):
            message.pretty_print()
        else:
            print(message)
else:
    print("No messages found in result.")

================================ Human Message =================================

Final Summary:
 # Meeting Summary - July 23, 2025  
**Attendees**: Alice, Bob, Charlie  

### 1. Project Phoenix  
- **Status Update**: Bob confirmed successful server deployment.  
- **Next Steps**: Action item identified for budget review (processed via tool).  

### 2. Marketing  
- **Summer Sale**: Launch scheduled for August 1st.  
- **Technical Prep**: Action item assigned to refine BERT chatbot for user intent analysis (processed via tool).  

All action items have been extracted and stored for tracking.

✅ Final Insights:
 **Key Themes**

* **Strategic Resource Allocation**: The meeting minutes hint at potential misalignment on departmental priorities, with discussions around budget reviews and resource allocation for Project Phoenix and the Summer Sale.

**Critical Decisions & Implications**

* **Project Phoenix Deployment**: Decision: The server has been deployed. Implication: This marks a signi